# Note
* pytraj/cpptraj use Amber mask for atom selection
* To compat with cpptraj output, pytraj use two types of indexing, 0-based and 1-based. Whenever using an integer, pytraj used 0-based index and whenever using a `string`, pytraj used 1-based index

In [1]:
import pytraj as pt

# load Topology file for demonstraction.
# if you download `pytraj` from github, you can find the below top file in $PYTRAJHOME/tests/data/
top = pt.load_topology("../tests/data/Tc5b.top")
traj = pt.load("../tests/data/md1_prod.Tc5b.x", top)

### Two types of indexing in pytraj

In [2]:
# 0-based indexing whenever using integer
top[0]

<N-atom, resnum=0, n_bonds=4>

In [3]:
# 1-based indexing whenever using `string` for atom mask
top['@1']

<Topology: 1 atoms, 1 residues, 1 mols, non-PBC>

## Atom Mask Selection in AMBER

(from Amber14 manual: http://ambermd.org/doc12/Amber14.pdf)

#### Amber Masks
A "mask" is a notation which selects atoms or residues for special treatment. A frequent usage is ﬁxing or
tethering selected atoms or residues during minimization or molecular dynamics.
The following lines are partially copied from the original AMBER documentation. For more details, refer to the
entire section of that documentation describing the ambmask utility.
The "mask" selection expression is composed of "elementary selections". *These start with ":" to select by
residues, or "@" to select by atoms.* Residues can be selected by numbers (given as numbers separated by commas,
or as ranges separated by a dash) or by names (given as a list of residue names separated by commas). The same
holds true for atom selections by atom numbers or atom names. In addition, atoms can be selected by AMBER
atom type, in which case "@" must be immediately followed by "%". The notation ":*" means all residues and
"@*" means all atoms. The following examples show the usage of this syntax.

Residue Number List Examples
```
:1-10 = "residues 1 to 10"
:1,3,5 = "residues 1, 3, and 5"
:1-3,5,7-9 = "residues 1 to 3 and residue 5 and residues 7 to 9"
```
Residue Name List Examples
````
:LYS = "all lysine residues"
:ARG,ALA,GLY = "all arginine and alanine and glycine residues"
```
Atom Number List Examples Note that these masks use the actual sequential numbers of atoms in the ﬁle.
This is tricky and a serious source of error. You must know these numbers correctly. Using the atom numbers of
a PDB ﬁle written out by an AMBER tool is an appropriate way to avoid pitfalls. Do not use the original atom
numbers from the raw PDB ﬁle you started with.
 
@12,17 = "atoms 12 and 17"
@54-85 = "all atoms from 54 to 85"
@12,54-85,90 = "atom 12 and all atoms from 54 to 85 and atom 90"
 
Atom Name List Examples  
@CA = all atoms with the name CA (i.e., all C-alpha atoms)
@CA,C,O,N,H = all atoms with names CA or C or O or N or H
(i.e., the entire protein backbone)
 

### let's try some example

``` python
    >>> top[':1-10'] # a list of atoms from residue 1 to 10
    >>> top[':1,3,5'] # a list of atoms in residue 1, 3, 5 (index starts from 1 when using string index)
    >>> top[[0, 2, 4]] # a list of atoms with indcies 0, 2, 4 (index starts from 0 when using integer index)
    >>> top['@CA'] # a list of CA atoms
    >>> top[':2-10@CA'] # a list of CA atoms from residue 2 to 10 (index starts from 1)
```

In [4]:
top[[0, 2, 4]]

[<N-atom, resnum=0, n_bonds=4>,
 <H2-atom, resnum=0, n_bonds=1>,
 <CA-atom, resnum=0, n_bonds=4>]

In [5]:
print (traj)
# get new Trajectory, keep only coords of residues 1 to 3, 5, 7 to 9 (index starts from 1)
t = traj[':1-3,5,7-9']
print (t)
print (t.top.residue_names)

<pytraj.Trajectory, 10 frames, include:
<Topology: 304 atoms, 20 residues, 1 mols, non-PBC>>
           
<pytraj.Trajectory, 10 frames, include:
<Topology: 126 atoms, 7 residues, 3 mols, non-PBC>>
           
{'ASP ', 'LEU ', 'TYR ', 'ASN ', 'LYS ', 'GLN '}


In [6]:
# all carbons except backbone alpha and carbonyl carbon
top['@C= & !@CA,C'][:5] # print only first 5 atoms

[<CB-atom, resnum=0, n_bonds=1>,
 <CG-atom, resnum=0, n_bonds=1>,
 <CB-atom, resnum=1, n_bonds=1>,
 <CG-atom, resnum=1, n_bonds=3>,
 <CD1-atom, resnum=1, n_bonds=1>]

In [7]:
# all SER and ARG atoms except those which are in residues 1-10 and which are CA or CB
top[':SER,ARG & !(:1-10 | @CA,CB)'][:5] # print only first 5 atoms

[<N-atom, resnum=0, n_bonds=1>,
 <H-atom, resnum=0, n_bonds=1>,
 <HA-atom, resnum=0, n_bonds=0>,
 <HB2-atom, resnum=0, n_bonds=0>,
 <HB3-atom, resnum=0, n_bonds=0>]

In [8]:
# all heavy atoms
new_top = top.strip_atoms('@H=', copy=True)
print (new_top.atom_names)

{'CZ3 ', 'ND2 ', 'OE1 ', 'OH  ', 'CZ2 ', 'CG  ', 'NZ  ', 'NE1 ', 'CE3 ', 'CZ  ', 'NH1 ', 'OXT ', 'NE2 ', 'CE1 ', 'C   ', 'OG  ', 'CD  ', 'OD1 ', 'CD2 ', 'CE  ', 'NE  ', 'OD2 ', 'CB  ', 'O   ', 'CA  ', 'CD1 ', 'NH2 ', 'N   ', 'CE2 ', 'CG2 ', 'CG1 ', 'CH2 '}


In [9]:
# all H in LYS
new_top = top.strip_atoms('!(:LYS@H=)', copy=True)
print (new_top.atom_names)
print (new_top.residue_names)
print (new_top)

{'HZ1 ', 'HZ3 ', 'H   ', 'HA  ', 'HB3 ', 'HE2 ', 'HE3 ', 'HG2 ', 'HB2 ', 'HD2 ', 'HD3 ', 'HZ2 ', 'HG3 '}
{'LYS '}
<Topology: 13 atoms, 1 residues, 13 mols, non-PBC>


In [10]:
# inplace-strip all H atoms
top.strip_atoms('@H=')
print (top.atom_names)

{'CZ3 ', 'ND2 ', 'OE1 ', 'OH  ', 'CZ2 ', 'CG  ', 'NZ  ', 'NE1 ', 'CE3 ', 'CZ  ', 'NH1 ', 'OXT ', 'NE2 ', 'CE1 ', 'C   ', 'OG  ', 'CD  ', 'OD1 ', 'CD2 ', 'CE  ', 'NE  ', 'OD2 ', 'CB  ', 'O   ', 'CA  ', 'CD1 ', 'NH2 ', 'N   ', 'CE2 ', 'CG2 ', 'CG1 ', 'CH2 '}


### what's about being interested only in atom indices?

In [11]:
# use () instead of []. [] is normally used for list, dictionary while () is used for callable function
top('@CA').indices

array([  1,   9,  17, ..., 137, 144, 148])

In [12]:
top('@CA') # return an AtomMask object to pass around.

### pytraj/cpptraj support distance-based mask selection too
```
    we need to load Frame object (as trajectory snapshot with xyz coords and other methods come with)
```

In [13]:
# load traj with trajectory filename and preloaded Topology
traj = pt.load("../tests/data/md1_prod.Tc5b.x", top)
traj

<pytraj.Trajectory, 19 frames, include:
<Topology: 154 atoms, 20 residues, 1 mols, non-PBC>>
           

In [14]:
# to use distance-based mask selction, we need to set_reference_frame
# example: set_reference_frame for the last frame
top.set_reference_frame(traj[-1])

# do the mask selection. Pick up all atoms within 5.0 Angstrom from given atom 1 (or 0 if using integer as indexing)

top("@1 <:5.0").indices

array([ 0,  1,  2, ..., 42, 43, 44])

In [15]:
# what can we do with atom mask?
# we can use AtomMask object to save new traj having only selected atoms

# save to AtomMask object to pass around
atm = top("@1 <@5.0")

# getting new Trajectory object having only selected atoms

print ("before")
print (traj)
print ("after")
new_traj = traj[atm]
print (new_traj)

before
<pytraj.Trajectory, 19 frames, include:
<Topology: 154 atoms, 20 residues, 1 mols, non-PBC>>
           
after
<pytraj.Trajectory, 19 frames, include:
<Topology: 23 atoms, 5 residues, 5 mols, non-PBC>>
           
